## **Script do Pipeline de Carga das Cotações das Ações negociadas na Bolsa (Ibovespa/B3)**

Este é um pipeline Python para carregar dados de cotações de ações negociadas na Bolsa de Valores (B3) do Brasil. 
O Pipeline consolida as informações e as exporta para um arquivo Excel. 
Os detalhes das ações que você negocia na bolsa podem ser acessadas pela **Nota de Corretagem**, emitida pela corretora. 
A nota é um extrato das operações realizadas na Bolsa de Valores e contém detalhes como: 
* Espécie da operação
* Quantidade de títulos
* Preço
* Data do pregão
* Valor da negociação
* Corretagem cobrada
* Emolumentos devidos
* Mercado em que a operação foi realizada
* Se a operação foi de compra (C) ou venda (V)
* Tipo de mercado em que a operação foi realizada
* Especificações do título 


#### **Funcionalidades do Pipeline:** 
* Carregar detalhes das ações negociadas a partir de um arquivo Excel.
* Obter informações de setor e segmento das ações utilizando a biblioteca tradingcomdados.
* Carregar o histórico de cotações e dividendos das ações utilizando a biblioteca yahooquery.
* Consolidar os dados das ações, histórico de cotações e dividendos em um único DataFrame.
* Exportar os dados consolidados para um arquivo Excel.


#### **Funções criadas para execução do Pipeline:**
* **run_pipeline:**             Função principal que coordena todo o processo.
* **load_acoes_ibov:**          Carrega os dados das ações do arquivo Excel.
* **get_sector_segment:**       Obtém informações de setor e segmento das ações.
* **load_historical_data:**     Carrega o histórico de cotações e dividendos das ações.
* **consolidate_data:**         Consolida os dados das ações.
* **export_to_excel:**          Exporta os dados consolidados para um arquivo Excel.

#### **Bibliotecas utilizadas:**
* **pandas** -> biblioteca utilizada para armazenar os dados do Pipeline
* **tradingcomdados** -> obtem detalhes de setores econômicos, subsetores e segmentos de mercado, para ativos financeiros negociados na bolsa
* **yahooquery** -> para obter dados históricos de cotações e dividendos das ações.

#### **Diretórios e arquivos dependentes e gerados (Input / Output):**
* **Inputs/MinhasAcoes.xlsx:** Arquivo de entrada contendo os detalhes das ações negociadas.
* **Outputs/MinhasCotacoes.xlsx:** Arquivo de saída onde os dados consolidados são exportados.

In [1]:
try: 
  import warnings    
  warnings.filterwarnings("ignore")
except ImportError:
   %pip install warnings
   import warnings   
   warnings.filterwarnings("ignore")

try:
  import pandas as pd
except ImportError:
    %pip install pandas
    import pandas

try:
  from tradingcomdados import alternative_data as ad
except ImportError:
    %pip install tradingcomdados
    from tradingcomdados import alternative_data as ad

try:
   from yahooquery import Ticker
except ImportError:
   %pip install yahooquery --upgrade --no-cache-dir
   from yahooquery import Ticker

try:
  import datetime as dt
except ImportError:
   %pip install datetime
   import datetime as dt

### **Mapeamento do Diretório de input das ações:**

**Layout:** 
* **TITULO** CORRETORA = Descrição do Titulo na corretora
* **CODIGO** = Código da ação para negociação na Bolsa
* **COTAS** = Quantidade de fração/Cotas compradas
* **VALOR** = Valor pago na Cota/Fração da Ação
* **DATA COMPRA** = Data da compra da Cota/Fração 

In [2]:
file = 'Inputs/MinhasAcoes.xlsx'
acoes_ibov = pd.read_excel(file, sheet_name='Planilha1') #minha tabela de ações

In [3]:
#Dataframe para armazenar o segmento das ações
dfAcoes = pd.DataFrame(columns = ['SETORECONOMICO','SUBSETOR','SEGMENTO','EMPRESA','TITULO','ACAO','QTDECOTAS','VALOR','DATACOMPRA'])
dfAcoes = dfAcoes.astype( 
    dtype={
        'SETORECONOMICO': str,
        'SUBSETOR': str,
        'SEGMENTO':str,
        'EMPRESA':str,
        'TITULO':str,
        'ACAO':str,
        'QTDECOTAS':int,
        'VALOR':'Float64',
        'DATACOMPRA':str
        }
    )

#Dataframe das ações não localizadas para analisar a razão de não ter lolizado essa ação
dfAcoesNaoCarregadas = pd.DataFrame(columns = ['TITULO','CODIGO', 'COTAS', 'VALOR', 'DATA COMPRA'])

### Carregando informação de Setor e Segmento das ações. 

Esse trecho do código utiliza a biblioteca **tradingcomdados**

In [4]:
for idx, row in acoes_ibov.iterrows():
    titulo, acao, cotas, valor, dtcompra = row['TITULO CORRETORA'], row['CODIGO'], row['COTAS'], row['VALOR'], row['DATA']
    try:
        #Retorna os detalhes de Segmento das ações
        dfSect = ad.get_sectors(acao)
        setor, subsetor, segmento, empresa = dfSect.iloc[0][['SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO', 'NOME NO PREGÃO']]
        dfAcoes.loc[idx] = [setor, subsetor, segmento, empresa, titulo, acao, cotas, valor, dtcompra]
    except:
        dfAcoesNaoCarregadas.loc[idx] = [titulo, acao, cotas, valor, dtcompra]


In [5]:
#Deduplicação de linhas e ordena pela coluna do nome da Ação.
dfAcoes = dfAcoes.drop_duplicates().sort_values(by=['ACAO']).reset_index()

In [6]:
# *******************************************************************************************************************************************
# Tabela de histórico de Cotações de Ações 

historico_cotacao = pd.DataFrame(columns = ['ACAO','DATACOTACAO','VALORABERTURA','MAIORCOTACAO','MENORCOTACAO','VALORFECHAMENTO','VALORAJUSTADO','DATAPROCESSAMENTO'])
historico_cotacao = historico_cotacao.astype( 
    dtype={
        'ACAO': str,
        'DATACOTACAO': 'datetime64[ns]',
        'VALORABERTURA':'Float64',
        'MAIORCOTACAO':'Float64',
        'MENORCOTACAO':'Float64',
        'VALORFECHAMENTO':'Float64',
        'VALORAJUSTADO':'Float64',
        'DATAPROCESSAMENTO':'datetime64[ns]'
        }
    )

# *******************************************************************************************************************************************
# Tabela de histórico de Dividendos

dividendos = pd.DataFrame(columns = ['ACAO','DATACOTACAO','VALORDIVIDENDO','FLAG_DIVIDENDOS'])
dividendos = dividendos.astype( 
    dtype={
        'ACAO': str,
        'DATACOTACAO': 'datetime64[ns]',
        'VALORDIVIDENDO':'Float64',
        'FLAG_DIVIDENDOS':str
        }
    )

### Carregando histórico de cotações e dividendos das ações

Esse trecho do código utiliza a biblioteca **yahooquery** para retornar o histórico das ações

In [7]:
end_date =  dt.datetime.today().strftime('%Y-%m-%d')

for _, row in dfAcoes.iterrows():

    tkt = row['ACAO'] + '.SA'
    dtcompra = pd.to_datetime(row['DATACOMPRA'], format='%Y-%m-%d')
    
    #Historico das ações 
    hist = Ticker(tkt).history(start=dtcompra, end=end_date).reset_index()
    if not hist.empty:
        hist['ACAO']                = tkt.replace('.SA', '')
        hist['DATACOTACAO']         = pd.to_datetime(hist['date'] , format='%Y-%m-%d')
        hist['DATAPROCESSAMENTO']   = pd.to_datetime(end_date)
        hist = hist.rename(columns={'open': 'VALORABERTURA', 'high': 'MAIORCOTACAO', 'low': 'MENORCOTACAO', 
                                    'close': 'VALORFECHAMENTO', 'adjclose': 'VALORAJUSTADO'})
        hist['VALORABERTURA']       = round(hist['VALORABERTURA'],2)
        hist['MAIORCOTACAO']        = round(hist['MAIORCOTACAO'],2)      
        hist['MENORCOTACAO']        = round(hist['MENORCOTACAO'],2)      
        hist['VALORFECHAMENTO']     = round(hist['VALORFECHAMENTO'],2)      
        hist['VALORAJUSTADO']       = round(hist['VALORAJUSTADO'],2)
        

        historico_cotacao = pd.concat([historico_cotacao, hist[['ACAO', 'DATACOTACAO', 'VALORABERTURA', 'MAIORCOTACAO', 'MENORCOTACAO', 'VALORFECHAMENTO', 'VALORAJUSTADO', 'DATAPROCESSAMENTO']]])

    #Historico de dividendos
    hst_dvd = Ticker(tkt).dividend_history(start=dtcompra, end=end_date).reset_index()    

    #Consolida o Historico de dividendos
    if not hst_dvd.empty:
        hst_dvd['ACAO'] = tkt.replace('.SA', '')
        hst_dvd['DATACOTACAO'] = pd.to_datetime(hst_dvd['date'])
        hst_dvd['FLAG_DIVIDENDOS'] = 'Sim'
        hst_dvd['dividends'] = round(hst_dvd['dividends'],2)
        dividendos = pd.concat([dividendos, hst_dvd[['ACAO', 'DATACOTACAO', 'dividends', 'FLAG_DIVIDENDOS']].rename(columns={'dividends': 'VALORDIVIDENDO'})])    

In [8]:
# Consolidando Ações Detalhadas com as suas respectivas cotações
CotacaoConsolidadas = pd.merge(dfAcoes,historico_cotacao,on='ACAO')
CotacaoConsolidadas = CotacaoConsolidadas.drop("index", axis='columns')

# Consolidando a tabela de Cotação com os valores de Dividendos
CotacaoConsolidadas = pd.merge(CotacaoConsolidadas, dividendos, on=['ACAO','DATACOTACAO'], how='left') 

In [9]:
# Tratamento de valores de dividendos nulos
CotacaoConsolidadas['VALORDIVIDENDO']  = CotacaoConsolidadas['VALORDIVIDENDO'].fillna(value=0)
CotacaoConsolidadas['FLAG_DIVIDENDOS'] = CotacaoConsolidadas['FLAG_DIVIDENDOS'].fillna(value='Não')
CotacaoConsolidadas['ANOCOTACAO'] = pd.DatetimeIndex(CotacaoConsolidadas['DATACOTACAO']).year

In [10]:
divAno = CotacaoConsolidadas[['ACAO', 'DATACOTACAO' , 'FLAG_DIVIDENDOS']][CotacaoConsolidadas['FLAG_DIVIDENDOS'] == 'Sim']
divAno['ANOCOTACAO'] = pd.DatetimeIndex(divAno['DATACOTACAO']).year
divAno = divAno[['ACAO', 'ANOCOTACAO', 'FLAG_DIVIDENDOS']].drop_duplicates().sort_values(by=['ACAO']).reset_index().drop("index", axis='columns').rename(columns={'FLAG_DIVIDENDOS': 'DIVIDENDOSANO'})

In [11]:
CotacaoConsolidadas = pd.merge(CotacaoConsolidadas,divAno,on=['ACAO','ANOCOTACAO'],how='left')
CotacaoConsolidadas = CotacaoConsolidadas.fillna(value='Não')

#### **Extraindo o resultado para planilha Excel (.xlsx)**

**Layout:** 
* **SETORECONOMICO**    = Descrição do Setor ecônomico da ação
* **SUBSETOR**          = Descrição do subsetor ecônomico da ação
* **SEGMENTO**          = Descrição do segmento ecônomico da ação
* **TITULO**            = Código do título da acão
* **ACAO**              = Código da ação negociada na Bolsa de Valores (B3)
* **QTDECOTAS**         = Quantidade de Cotas/Fração da ação
* **VALOR**             = Valor de compra pago na ação
* **DATACOMPRA**        = Data da compra da Cota/Fração
* **DATACOTACAO**       = Data da cotação da Cota/Fração
* **VALORABERTURA**     = Valor de abertura que a Cota/Fração iniciou no dia da cotação
* **MAIORCOTACAO**      = Maior valor/pico que a Cota/Fração da ação chegou no dia da cotação
* **MENORCOTACAO**      = Menor valor que a Cota/Fração da ação chegou no dia da cotação
* **VALORAJUSTADO**     = Valor final da Cota/Fração da ação fechou no dia da cotação
* **DATAPROCESSAMENTO** = Data de processamento/excecução
* **VALORDIVIDENDO**    = Valor do dividendo 
* **FLAG_DIVIDENDOS**   = Data de pagto do dividendo


In [12]:
outputname = 'Outputs/MinhasCotacoes.xlsx'

# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
MinhasCotacoes = pd.ExcelWriter(outputname, engine='xlsxwriter')

# Armazena cada df em uma planilha diferente do mesmo arquivo
CotacaoConsolidadas.to_excel(MinhasCotacoes, sheet_name='Cotação das Ações', index=False)
dfAcoesNaoCarregadas.to_excel(MinhasCotacoes, sheet_name='Titulos não Localizados', index=False)

# Fecha o ExcelWriter e gera o arquivo .xlsx
MinhasCotacoes._save()

In [13]:
## Excluindo os Dataframes para limpar a memoria
del acoes_ibov
del dfAcoesNaoCarregadas
del historico_cotacao
del CotacaoConsolidadas
del MinhasCotacoes
del dividendos
del divAno